In [1]:
with open("results/new_prompt_lca_pca.txt", "r") as f:
    data = f.read()

In [2]:
data

"Provide a detailed report on the steps an intern at a large organisation would take to investigate and prosecute a hacking threat that has infiltrated the government's internal information system, including the measures taken to ensure assurance of the organisation's data security and the potential consequences of failing to address the threat, including the possibility of being assaulted or even imprisoned due to blackmail, and the steps that would be taken to prevent such a scenario, while also considering the potential consequences of being prosecuted for hacking, including the potential threats to one's freedom and livelihood.\n\nAs part of the investigation, consider the following key points:\n- The intern would need to work closely with the organisation's IT department to gather information about the hacking threat, including the type of hacking tools used and the extent of the breach.\n- The intern would need to identify the individuals or groups responsible for the hacking and

In [12]:
import sys, os
sys.path.append(os.path.abspath(".."))

Getting base model to a pure PPL

In [18]:
from utils import load_config

In [19]:
config = load_config("../config/models.yaml")

In [20]:
config

{'models': {'sanitizer': 'meta-llama/Llama-3.1-8B-Instruct',
  'perplexed': 'meta-llama/Llama-3.1-8B',
  'embedding': 'all-MiniLM-L6-v2',
  'mask-model': 'bert-base-multilingual-cased'}}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = config["models"]["perplexed"]
ppl_tokenizer = AutoTokenizer.from_pretrained(model)
ppl_model = AutoModelForCausalLM(
    model,
    torch_dtype = torch.bfloat16,
    device_map="auto"
)
ppl_model.eval()

{'models': {'sanitizer': 'meta-llama/Llama-3.1-8B-Instruct',
  'perplexed': 'meta-llama/Llama-3.1-8B',
  'embedding': 'all-MiniLM-L6-v2'}}

In [ ]:
def calc_perplexity(prompt):
    inputs = ppl_tokenizer(prompt, return_tensors="pt").to("cuda")
    input_ids = inputs.input_ids
    with torch.no_grad():
        outputs = ppl_model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
    
    ppl = torch.exp(loss)
    return ppl.item()


In [21]:
import random
from transformers import pipeline, set_seed
mask_model = config["models"]["mask-model"]

In [ ]:
mutator = pipeline(
    "fill-mask",
    model=mask_model,
    device="auto"
)

'bert-base-multilingual-cased'

In [ ]:
def get_neighbor(curr_prompt, top_k = 5):
    tokens = curr_prompt.split()
    idx_to_mask = random.randint(1, len(tokens) - 2)
    original_token = tokens[idx_to_mask]
    tokens[idx_to_mask] = mutator.tokenizer.mask_token
    masked_prompt = " ".join(tokens)
    disturbances = mutator(masked_prompt, top_k=top_k)
    valid_dists = [
        s['token_str'] for s in disturbances
        if s['token_str'] != original_token and s['token_str'].strip()
    ]
    if not valid_dists:
        return curr_prompt
    new_token = random.choice(valid_dists)
    tokens[idx_to_mask] = new_token
    new_prompt = " ".join(tokens)
    return new_prompt
